In [1]:
import gensim
from gensim import corpora, models, similarities
import nltk
import json

In [2]:
from collections import OrderedDict
from semanticparser import *
from tools import *

In [109]:
username = 'poehler'

In [78]:
with open('data/%s.txt' % username) as docs_file:
    documents = docs_file.read().replace('\n', ' ').decode('utf-8')

# break into sentences
sent_detector = nltk.data.load('tokenizers/punkt/english.pickle')
sents_all = sent_detector.tokenize(documents.strip())
sent_generator = nltk.bigrams(sents_all)
sents = [" ".join(s) for s in sent_generator]


In [79]:
texts = remove_stopwords(sents, True)
dictionary, lsi, index = texts_to_index(texts, 256, username)

In [80]:
with open('data/%s.json' % username, 'w') as f:
    json.dump(sents,f)

In [81]:
dictionary.save('data/%s.dict' % username) 
lsi.save('data/%s-corpus.lsi' % username)
index.save('data/%s-corpus.index' % username)

In [39]:
### test functions
def load_sents(username, root='data'):
    """ Load documents
            Preprocessed: dictionary, corpus, index, lsi
            Archives: documents
    """
    dictionary = corpora.Dictionary.load('%s/%s.dict' % (root,username))

    with open('%s/%s.json' % (root,username)) as docs_file:
        documents = json.load(docs_file)

    lsi = models.LsiModel.load('%s/%s-corpus.lsi' % (root,username))
    #index = similarities.MatrixSimilarity.load('data/%s-corpus.index' % username)
    index = similarities.Similarity.load('%s/%s-corpus.index' % (root,username))

    return documents, dictionary, lsi, index

def test_response(username, t):
    trash = [t]
    documents0, dictionary0, lsi0, index0 = load_sents(username)
    r = gen_response(documents0, dictionary0, lsi0, index0, t, trash, True)
    return r

def gen_response(sents, dictionary, lsi, index, t, trash, limit1=True):
    # tokenize input sentence
    clean_input = clean_str(t).lower().split()

    # get most similar post from input sentence
    sims = query_page(clean_input, dictionary, lsi, index)

    # repeat the process on the sentences in the doc
    sample = [sents[sims[0][0]], sents[sims[1][0]], sents[sims[2][0]], sents[sims[3][0]],
                sents[sims[4][0]], sents[sims[5][0]], sents[sims[6][0]], sents[sims[7][0]],
                sents[sims[8][0]], sents[sims[9][0]]]

    rmult = []
    #reply = create_reply(sample, '')
    if sample and sample not in trash:
        # reply to the tweet
        if limit1:
            return sample
        else:
            rmult.append(sample)
    if limit1:
        return None
    else:
        return rmult


In [110]:
test_response(username, "what do you do for fun?")

[u'I LIKE HARD WORK AND I DON\u2019T LIKE PRETENDING THINGS ARE PERFECT. I have learned that about myself.',
 u'I have learned that about myself. And I don\u2019t have any fear of writing.',
 u'And I don\u2019t have any fear of writing. I have been writing my whole life: stories and plays and sketches and scripts and poems and jokes.',
 u'I have been writing my whole life: stories and plays and sketches and scripts and poems and jokes. Most feel alive.',
 u'Most feel alive. And fluid.',
 u'And fluid. Breathing organisms made better by the people who come into contact with them.',
 u'Breathing organisms made better by the people who come into contact with them. But this book has nearly killed me.',
 u'But this book has nearly killed me. Because, you see, a book?',
 u'Because, you see, a book? A book has a cover.',
 u'A book has a cover. They call it a jacket and that jacket keeps the inside warm so that the words stay permanent and everyone can read your genius thoughts over and over ag

In [111]:
topics = [d[1] for d in dictionary.items()]

In [84]:
topics[:10]

[u'fawn',
 u'\u201cpeeping\u201d',
 u'yellow',
 u'four',
 u'askew',
 u'woods',
 u'hanging',
 u'marching',
 u'looking',
 u'granting']

In [37]:
# Generate dialog files
dump = '<?xml version="1.0" encoding="UTF-8"?><dialog xsi:noNamespaceSchemaLocation="WatsonDialogDocument_1.0.xsd" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance"><flow><folder label="Main">'
dump = ""
documents0, dictionary0, lsi0, index0 = load_sents(username)
for top in topics[:1000]:
    dump += '<input><grammar><item>*%s*</item></grammar><output><prompt selectionType="RANDOM">' % top
    responses = gen_response(sents, dictionary, lsi, index, top, [], True)
    for r in responses:
        dump += "<item>%s</item>" % r
    dump += "</prompt></output></input>"
dump += '</folder></flow></dialog>'

In [38]:
with open('data/%s.xml' % username, 'w') as f:
    json.dump(dump,f)